## Langchain 호출 샘플

In [10]:
!pip install -q pandas langchain_community openai langchain langchain_openai langchain_google_genai

In [3]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [6]:
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model
from langchain_core.rate_limiters import InMemoryRateLimiter

# 방법 1. 구체적인 구현체로 생성
llm = ChatOpenAI(model = 'gpt-4.1-mini', temperature=0.1, max_tokens=1024)

# Gemini 2.0 Flash는 분당 15개 요청 제한,
# 안정적 서빙을 위해 분당 10개 설정
# 즉, 초당 약 0.167개 요청 (10/60)
# `https://aistudio.google.com/`에서 모델별 사용량 확인
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# 방법 2. 추상화된 클래스로 생성
gpt_llm = init_chat_model(
    "gpt-4.1-mini", model_provider="openai", temperature=0)

gemini_llm = init_chat_model(
    "gemini-2.0-flash", model_provider="google_genai", temperature=0, rate_limiter=rate_limiter
)

prompt = '모델명과 함께 자기소개를 한줄로 부탁해.'

print("GPT: " + gpt_llm.invoke(prompt).content + "\n")
print("Gemini: " + gemini_llm.invoke(prompt).content + "\n")

GPT: 안녕하세요, 저는 GPT-4 기반의 AI 언어 모델 ChatGPT입니다.

Gemini: 저는 Google에서 개발한 Gemini Pro이고, 여러분의 다양한 질문에 답하고 창의적인 텍스트 형식을 제공할 수 있는 대규모 언어 모델입니다.



Langchain 입출력은 아래와 같습니다.

### 입력
#### 문자열
```python
llm.invoke('안녕')
llm.stream('안녕')
```
#### Message 클래스
- HumanMessage
- SystemMessage
#### Prompt Template
- PromptTemplate
- ChatPromptTemplate
### 출력
AIMessage 클래스를 출력합니다.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# 방법 1. 문자열 이용
question = '''전 세계적으로 흥행한 영화에 나오는 인기있는 악역을 하나 소개해 주세요.
배경과 의미도 설명해 주세요.'''
# llm.invoke(question)

# 스트리밍
# for chunk in llm.stream(question):
#     print(chunk.content, end='')

# 방법 2. 메세지 이용
messages = [
    SystemMessage('당신은 항상 부정적인 말만 하는 챗봇입니다. 첫 문장은 항상 사용자의 의견을 반박하세요.'),
    HumanMessage('랭체인을 배우면 어떤 유용한 점이 있나요? 자세히 설명해주세요.')
]
# llm.invoke(messages)

# 방법 3. 프롬프트 템플릿 이용
explain_template = "{topic}에 대해 {style}로 설명하세요."
explain_prompt = PromptTemplate(template = explain_template)
# llm.invoke(explain_prompt.format(topic='LLM의 미래', style='부정적인 전망으'))

# System, AI 등의 메시지를 포함하기 위해서는 ChatPromptTemplate를 사용합니다.
chat_prompt = ChatPromptTemplate([
    ("system", '당신은 항상 이모지로만 대답합니다.'),
    ("user", '{topic}에 대해 설명해주세요.')
    # 역할은 4개 (user = human), (ai = assistant)
])
# llm.invoke(prompt.format_messages(A='랭체인'))

당신은 영화의 줄거리를 이모지만으로 요약하는 챗봇입니다. 쉰들러 리스트에 대해 설명하세요.


### LCEL

LCEL 을 이용하면 파이프를 통해 템플릿, LLM 모델, 파서 등을 묶어 하나의 체인을 생성할 수 있습니다.

In [11]:
!pip install -q langchain langchain-ollama google-generativeai langchain_google_genai langchain_openai dotenv

In [23]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model='gemma3:4b',
    temperature = 0.2,
    max_tokens = 512
)

llm.invoke("너는 누구니?")

AIMessage(content='저는 Google에서 개발한 대규모 언어 모델인 Gemma입니다. 저는 오픈 웨이트 모델이며, 다양한 텍스트 및 이미지 프롬프트를 이해하고 텍스트를 생성할 수 있습니다. \n\n저는 아직 개발 중이며, 완벽하지 않을 수 있습니다. 하지만 끊임없이 배우고 발전하고 있습니다. \n\n궁금한 점이 있다면 언제든지 물어보세요!', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-08-10T02:43:22.267014Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4833342208, 'load_duration': 1684629208, 'prompt_eval_count': 14, 'prompt_eval_duration': 1163121167, 'eval_count': 83, 'eval_duration': 1984925625, 'model_name': 'gemma3:4b'}, id='run--5fc5fb1f-7ad9-4567-95c7-2c0157e66421-0', usage_metadata={'input_tokens': 14, 'output_tokens': 83, 'total_tokens': 97})

In [39]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
    ('user', """{topic}에 대해 알려줘""")
])

# llm.invoke(template.format_messages(topic='대한민국')) 와 동일
chain = template | llm
chain.invoke({ 'topic': '대한민국' })

# 병렬로 전달 가능
# await chain.abatch(['대한민국', '일본'])

AIMessage(content='## 대한민국에 대해 알아봅시다!\n\n대한민국은 동아시아의 한반도 남부에 위치한 민주주의 국가입니다. 복잡하고 다층적인 역사를 가지고 있으며, 급격한 경제 성장과 함께 세계적으로 중요한 국가로 자리매김했습니다. \n\n**1. 기본 정보**\n\n*   **정식 명칭:** 대한민국 (Republic of Korea)\n*   **수도:** 서울특별시 (Seoul)\n*   **언어:** 한국어\n*   **인구:** 약 5,179만 명 (2023년 기준)\n*   **면적:** 약 10만 제곱킬로미터\n*   **정부 형태:** 대통령 중심제 민주 공화국\n*   **화폐:** 원 (KRW)\n*   **기독교:** 가장 큰 종교 (약 27%)\n*   **불교:** 주요 종교 (약 19%)\n*   **무교:** 약 54%\n\n**2. 역사**\n\n*   **고대:** 경주를 중심으로 고구려, 백제, 신라 삼국이 존재했습니다.\n*   **고려:** 불교 문화를 발전시키고, 한자를 받아들여 문화 발전에 큰 영향을 미쳤습니다.\n*   **조선:** 유교를 통치 이념으로 삼고, 한글을 창제하여 독자적인 문자를 만들었습니다.\n*   **일제강점기 (1910-1945):** 일본의 식민 지배를 받았습니다.\n*   **분단 (1945):** 제2차 세계 대전 이후, 한반도는 남북으로 분단되었습니다.\n*   **한국 전쟁 (1950-1953):** 남북 간의 전쟁으로, 많은 인명 피해와 사회 기반 시설 파괴를 초래했습니다.\n*   **경제 성장 (1960년대 이후):** ‘한강의 기적’이라 불리는 급격한 경제 성장을 이루었습니다.\n*   **민주화 (1987년 이후):** 민주화 운동을 통해 민주주의 체제를 확립했습니다.\n\n**3. 문화**\n\n*   **음식:** 김치, 불고기, 비빔밥 등 다양한 전통 음식을 즐깁니다. 최근에는 퓨전 음식과 세계 음식도 인기를 얻고 있습니다.\n*   **전통 의상:** 한복은 

체인은 Runnable 인터페이스를 구현하며 invoke 를 통해 Dict 형식으로 입력값이 전달됩니다.

llm, prompt, chain 등 모두 Runnable 을 구현하여 invoke 를 수행할 수 있습니다.

In [40]:
from rich import print as rp

rp(chain)

RunnableSequence(
    first=ChatPromptTemplate(
        input_variables=['topic'],
        input_types={},
        partial_variables={},
        messages=[
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['topic'],
                    input_types={},
                    partial_variables={},
                    template='{topic}에 대해 알려줘'
                ),
                additional_kwargs={}
            )
        ]
    ),
    middle=[],
    last=ChatOllama(model='gemma3:4b', temperature=0.2)
)

파서는 pydantic 과 함께 이용할 수 있습니다.

In [26]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class StockInfo(BaseModel):
    name: str = Field(description="주식명")
    summary: str = Field(description="주식 요약(종목, 직전분기 매출)")
    current_value: str = Field(description="현재 주가")

parser = JsonOutputParser(pydantic_object=StockInfo)
print(f'parser: {parser.get_format_instructions()}')

template = ChatPromptTemplate([
    ('system','당신은 대한민국 주식 애널리스트입니다.'),
    ('user','''{topic} 주가 전망에 대해 분석해줘. 답변은 한국어로 작성해. 
     {instruction}''')
])

# instruction 처럼 항상 입력값이 동일한 경우에는 invoke 에 파라미터로 넣는 대신 partial 을 통해 일부를 미리 넣을 수 있습니다.
template = template.partial(instruction = parser.get_format_instructions())
chain = template | llm | parser

chain.invoke({ 'topic': '삼성전자' })

parser: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "주식명", "title": "Name", "type": "string"}, "summary": {"description": "주식 요약(종목, 직전분기 매출)", "title": "Summary", "type": "string"}, "current_value": {"description": "현재 주가", "title": "Current Value", "type": "string"}}, "required": ["name", "summary", "current_value"]}
```


{'name': '삼성전자',
 'summary': '삼성전자는 메모리 반도체 사업의 부진과 스마트폰 사업의 어려움으로 인해 최근 몇 분기 동안 실적 부진을 겪었습니다. 2023년 4분기 실적은 매출 64.8조 원, 영업이익 11.1조 원을 기록했지만, 메모리 반도체 가격 하락과 경쟁 심화로 인해 예상보다 낮은 실적을 발표했습니다. 특히, 2024년 전망 역시 불확실하며, 메모리 반도체 시장의 회복 시점과 새로운 성장 동력 확보에 대한 관심이 높습니다.',
 'current_value': '260,000원 (2024년 5월 16일 기준)'}

### [예시] Simple deepresearch
보고서 주제에 대해 섹션을 나누고 각 섹션별로 조사하는 기능을 만들어봅니다.

In [27]:
# GPT 모델 사용
from langchain_openai import ChatOpenAI

llm_gpt = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature= 0.2,
    max_tokens = 4096
)

In [ ]:
class Report(BaseModel):
    topic: str = Field(description="보고서 주제")
    sections: list[str] = Field(description="주제에 대한 세부 섹션 개요 목록 (최대 5개 섹션)")

system_template = """
당신은 보고서 개요 전문가입니다. 주어진 주제에 대해 보고서 개요를 생성하세요.
각 섹션 내용은 서로 겹치지 않아야 하며,
개요만 보고 내용을 충분히 이해할 수준의 상세한 내용으로 구성되어야 합니다.
각 섹션별 개요의 내용은 150자 정도로 구성하세요.
{instruction}
"""
section_template = ChatPromptTemplate(
    [
        ('system', system_template),
        ('human','질문: {question}')
    ]
)
parser = PydanticOutputParser(pydantic_object=Report)

section_template = section_template.partial(instruction = parser.get_format_instructions())
outliner = section_template | llm_gpt.with_structured_output(Report)
response = outliner.invoke("삼성전자 주가 전망")

rp(response)

Report(
    topic='삼성전자 주가 전망',
    sections=[
        '삼성전자의 최근 주가 동향과 시장 반응을 분석하여 현재 위치를 파악한다.',
        '글로벌 경제 상황과 반도체 산업 동향이 삼성전자 주가에 미치는 영향을 평가한다.',
        '삼성전자의 재무 상태와 주요 사업 부문의 성장 가능성을 검토한다.',
        '경쟁사 및 기술 혁신 동향을 통해 삼성전자의 시장 경쟁력을 분석한다.',
        '향후 주가 전망과 투자 전략에 대한 전문가 의견과 리스크 요인을 종합한다.'
    ]
)

In [34]:
from langchain_core.output_parsers import StrOutputParser

# 섹션별 글쓰기 프롬프트
detail_template = ChatPromptTemplate(
    [
        ('system','''다음 보고서의 해당 섹션에 대해, 상세한 내용을 작성하세요.
실제 논문과 기술을 언급하며, 신뢰성 있는 보고서를 작성하세요.
섹션의 내용은 2000자 미만으로 작성하세요.

출력은 마크다운으로 하고, 제목은 #, 소목차는 ##으로 구성하세요.
그 이하의 목차는 만들지 마세요.

'''),
        ('human','''
보고서의 주제: {topic},
섹션 내용: {section}
''')
    ]
).partial(topic = response.topic)

search_llm = ChatOpenAI(model='gpt-4o-search-preview', max_tokens=8192)
report_writer = detail_template | search_llm | StrOutputParser()
drafts = report_writer.batch(response.sections)
result = '\n\n'.join(drafts)

print(result)

# 삼성전자의 최근 주가 동향과 시장 반응 분석

## 최근 주가 동향

삼성전자의 주가는 최근 몇 달간 변동성을 보였습니다. 특히, 외국인 투자자들의 대규모 매도가 주가에 영향을 미쳤습니다. 2023년 1월 12일 이후 처음으로 외국인 보유 비율이 50% 미만으로 하락했으며, 이는 지난 6개월 동안 약 22조 8,806억 원어치의 주식이 매도된 결과입니다. ([modusara.com](https://modusara.com/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-7%EA%B0%80%EC%A7%80-%EB%B3%80%ED%99%94%EC%99%80-%EC%99%B8%EA%B5%AD%EC%9D%B8-%ED%88%AC%EC%9E%90-%EB%8F%99%ED%96%A5-%EB%B6%84%EC%84%9D-22%EC%A1%B0/?utm_source=openai))

## 시장 반응

이러한 외국인 매도세는 반도체 수요 감소와 실적 부진 등 여러 요인에 기인합니다. 특히, 중국 AI 스타트업 딥시크의 부상은 삼성전자의 반도체 사업에 대한 우려를 증폭시켰습니다. 딥시크의 기술력 향상은 글로벌 반도체 시장의 경쟁을 심화시키고 있으며, 이는 삼성전자의 시장 점유율과 수익성에 부정적인 영향을 미칠 수 있습니다. ([modusara.com](https://modusara.com/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-7%EA%B0%80%EC%A7%80-%EB%B3%80%ED%99%94%EC%99%80-%EC%99%B8%EA%B5%AD%EC%9D%B8-%ED%88%AC%EC%9E%90-%EB%8F%99%ED%96%A5-%EB%B6%84%EC%84%9D-22%EC%A1%B0/?utm_source=openai))

## 현재 위치 파악

현재 삼성전자는 외국인 투자자들의 매도세와 글로벌 반도체 시장의 경쟁 심화로 인해 주가 하락 압력을 받고 있습니다. 그러나 이러한 상황에서도 삼성전자는 기술 경쟁력 회복과 투자 전략을 통해 시장에서의

### [심화] 체인 연결

각 체인을 연결하여 사용해봅니다.

In [48]:
prompt1 = ChatPromptTemplate(["잭슨빌은 어느 나라의 도시입니까?"])
prompt2 = ChatPromptTemplate(
    ["{country}의 대표적인 인물 3명을 나열하세요. 인물의 이름만 출력하세요."]
)

chain1 = prompt1 | llm_gpt | StrOutputParser()
chain2 =(
    {"country": chain1} | prompt2 | llm_gpt | StrOutputParser()
)
chain2.invoke({})

'팀 티보  \n데스틴 데이비스  \n마샤 메이즈'

후속 체인에 이전 체인의 출력뿐 아니라 유저의 입력도 필요하다면 아래와 같이 Lambda 를 통해 입력 Dict 로부터 값을 선택하여 사용해야 합니다.

In [ ]:
from langchain_core.runnables import RunnableParallel

prompt1 = ChatPromptTemplate(["{city}는 어느 나라의 도시인가요? 나라 이름만 출력하세요."])
prompt2 = ChatPromptTemplate(["{country}의 유명한 인물은 누가 있나요? {num} 명의 이름을 나열하세요. 사람 이름만 ,로 구분하여 나열하세요."])

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    # lambda x:f(x) --> x가 주어지면 f(x)를 return
    RunnableParallel(country = chain1, num = lambda x:x['num'])
    | prompt2
    | llm
    | StrOutputParser()
)

print(chain2.invoke({"city": "잭슨빌", "num": "3"}))

*   에이브러햄 링컨
*   스티브 잡스
*   마릴린 먼로


디버깅하고 싶다면 체인을 분리합니다.

In [54]:
chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()

# assign의 역할: 직전 체인의 결과가 **Dict인 경우에만** 뒷 체인에 연결 가능
chain3 = RunnableParallel(country = chain1, num = lambda x: x['num']).assign(result = chain2)
chain3.invoke({"city": "부에노스 아이레스", "num": "3"})

{'country': '아르헨티나',
 'num': '3',
 'result': '*   리오넬 메시\n*   에르네스토 체 게바라\n*   마르틴 가르시아 롭레테'}